In [1]:
import cv2
import numpy as np
import face_recognition
from PIL import Image
import glob
import re
import ntpath
import os
import dlib
import time
from imutils.face_utils import FaceAligner
import sqlite3
from imutils.face_utils import rect_to_bb
import imutils
from imutils import face_utils
import os
import shutil
import inception_resnet_v1
import tensorflow as tf

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
shutil.rmtree("../data/unknown_people/")

In [3]:
os.mkdir("../data/unknown_people/")

In [4]:
people = sqlite3.connect('people.db')
c = people.cursor()

In [5]:
# show name people know name
for row in c.execute('select * from people_known order by old'):
    print(row)

('Phạm Minh Anh', 0, 9)
('Dam Thi Phuong Linh', 0, 11)
('Dam Van Tai', 1, 24)
('vietnam', 1, 24)
('ha thi quynh huong', 1, 24)
('Pham Van Tuan', 1, 30)
('dam van tai', 1, 40)


In [6]:
# show name people in room equal empty
c.execute('delete from people_inroom')
people.commit()
for row in c.execute('select * from people_inroom order by old'):
    print(row)

In [7]:
# show id people unknown name
c.execute('delete from people_unknown')
people.commit()
for row in c.execute('select * from people_unknown order by old'):
    print(row)

In [8]:
# recognition face center
def face_center(image):
    image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    rects = detector(image_gray, 1)
    for (i, rect) in enumerate(rects):
        points = predictor(image_gray, rect)
        points = face_utils.shape_to_np(points)
        center_face = points[37][1] - points[46][1]
        if center_face < 10 & center_face > -10:
            return True
        else:
            return False

In [9]:
# sess = tf.Session()
# images_pl = tf.placeholder(tf.float32, shape=[None, 160, 160, 3], name='input_image')
# images_norm = tf.map_fn(lambda frame: tf.image.per_image_standardization(frame), images_pl)
# train_mode = tf.placeholder(tf.bool)
# age_logits, gender_logits, _ = inception_resnet_v1.inference(images_norm,
#                                                              weight_decay=1e-5)
# gender = tf.argmax(tf.nn.softmax(gender_logits), 1)
# age_ = tf.cast(tf.constant([i for i in range(0, 101)]), tf.float32)
# age = tf.reduce_sum(tf.multiply(tf.nn.softmax(age_logits), age_), axis=1)
# init_op = tf.group(tf.global_variables_initializer(),
#                    tf.local_variables_initializer())
# sess.run(init_op)
# saver = tf.train.Saver()
# ckpt = tf.train.get_checkpoint_state("../gender_age_tf/models/")
# if ckpt and ckpt.model_checkpoint_path:
#     saver.restore(sess, ckpt.model_checkpoint_path)
#     print("restore model!")
# else:
#     pass

In [10]:
# 

In [11]:
# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True
name = ""
frame_number = 0
total_in_room = 0
total_known_in_room = 0
index = 0

# Load known_face_encoding_array and known_face name from npy
known_face_encodings_array = np.load("../data/numpy/known_face_encoding.npy")
known_face_names = np.load("../data/numpy/known_face_names.npy")

# Convert nparray -> list 
number_person = len(known_face_encodings_array)
known_face_encodings_array = known_face_encodings_array.reshape(number_person, 128)
known_face_encodings = []
for i in range(len(known_face_encodings_array)):
    known_face_encodings.append(known_face_encodings_array[i])

# face encodings of people in room
face_encodings_in_room = []
face_encodings_unknown_in_room = []
face_names_unknown_in_room = []
# known_face_encodings: array known_face_encodings 

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
fa = FaceAligner(predictor, desiredFaceWidth=160)

# Capture from camera of own computer
video_capture = cv2.VideoCapture(0)

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Resize frame of video to 1/4 size for faster face recognition processing
    # Frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    frame_number += 1
    
    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = frame[:, :, ::-1]
        
    # Use computer configuration low
#     if (frame_number % 10 == 0):
    # Only process every other frame of video to save time
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
#         print(face_locations)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            face_unknown = frame[top:bottom, left:right]
            if face_center(face_unknown) == True:
                # See if the face is a match for the known face(s)
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.5)

    #             print(face_recognition.face_distance(known_face_encodings, face_encoding))
    #             print(matches)
    #             print(np.min(face_recognition.face_distance(known_face_encodings, face_encoding)))

                distance = face_recognition.face_distance(known_face_encodings, face_encoding)
                point = np.min(distance)
    #             print(distance)
                index_point_min = np.argmin(distance)
    #             print("gia tri cua index_point_min", index_point_min)
                if point > 0.5:
                    if os.path.isfile("../data/unknown_people/unknown_0.jpg"):
                        distance_unknown = face_recognition.face_distance(face_encodings_unknown_in_room, face_encoding)
                        min_distance_unknown = np.min(distance_unknown)
                        if min_distance_unknown > 0.6:
                            a = os.listdir("../data/unknown_people/")
                            a.sort()
                            index = int(a[-1].split('.')[0].split('_')[1])
                            index += 1
                            print("gia tri index", index)
                            path = "../data/unknown_people/unknown_" + str(index) + ".jpg"
                            name = "unknown_" + str(index)
                            cv2.imwrite(path, face_unknown)
                            face_encodings_unknown_in_room.append(face_encoding)
                            face_names_unknown_in_room.append(name)
                            total_in_room += 1
                            print(name)
                    else:
                        name = "unknown_0"
                        path = "../data/unknown_people/unknown_0" + ".jpg"
                        cv2.imwrite(path, face_unknown)
                        face_encodings_unknown_in_room.append(face_encoding)
                        face_names_unknown_in_room.append(name)
                        total_in_room += 1
                        print(name)

                # If a match was found in known_face_encodings, just use the first one.
                if True in matches:
    #                 first_match_index = matches.index(True)
    #                 print("gia tri cua first_match_index", first_match_index)
    #                 first_match_index = matches[index_point_min]
    #                 name = known_face_names[first_match_index]
                    name = known_face_names[index_point_min]
                    print(name)
                    c.execute("select * from people_known where name= '%s'" % name)
                    data_temp = c.fetchone()  # name in people known
                    c.execute("select * from people_inroom where name='%s'" % name)
                    data_name_inroom = c.fetchone() # name in room
                    if data_name_inroom == None:
                        c.execute("insert into people_inroom values (?, ?, ?)", data_temp)
                        people.commit()
                        total_in_room += 1
                        total_known_in_room += 1
                face_names.append(name)
#             print(name)


    # Display the results
    i = 0
    for (top, right, bottom, left), name, face_encoding_test in zip(face_locations, face_names, face_encodings):
        # save name
#         first_name = name.split("_")[0]
        
#         face_unknown = frame[top:bottom, left:right]
        
#         if face_center(face_unknown) == True:
#             if first_name != "unknown":
#                 c.execute("select * from people_known where name= '%s'" % first_name)
#                 data_temp = c.fetchone()  # name in people known
#                 c.execute("select * from people_inroom where name='%s'" % first_name)
#                 data_name_inroom = c.fetchone() # name in room
#                 # if people in room
#                 if data_name_inroom == None:
#                     c.execute("insert into people_inroom values (?, ?, ?)", data_temp)
#                     people.commit()
#                     total_in_room += 1
#                     total_known_in_room += 1
    #             else:
    #                 # name of that people is already in room
    #                 break
#             else:
#                 face_estimation_age_gender_gray = cv2.cvtColor(face_unknown, cv2.COLOR_BGR2GRAY)
# #                 face_estimation_age_gender = fa.align()
                
#     #             face_unknown = frame[top:bottom, left:right]
#     #             face_encoding_unknown = face_recognition.face_encodings(rgb_small_frame, face_locations[i])
#                 if len(face_encodings_unknown_in_room) == 0:
#     #                 point = np.min(face_recognition.face_distance(known_face_encodings, face_encoding_test))
#     #                 if point < 0.3:
#                     print("buoc 1")
#                     face_encodings_unknown_in_room.append(face_encoding_test)
#                     total_in_room += 1
#                     path = "../data/unknown_people/" + name + ".jpg"
#                     cv2.imwrite(path, face_unknown)
#                     # tim tuoi va gioi tinh cua nguoi do va luu va danh sach nguoi trong phong
#                     c.execute("insert into people_inroom values ('%s', 1, 12)" % name)
#                     people.commit()
#                 else:
#                     point = np.min(face_recognition.face_distance(face_encodings_unknown_in_room, face_encoding_test))
#                     # neu mà giống thì point càng thấp, nếu mà khác nhau 
#                     if point > 0.6:
#                         print("buoc 2")
#                         print(point)
#                         face_encodings_unknown_in_room.append(face_encoding_test)
#                         total_in_room += 1
#                         path = "../data/unknown_people/" + name + ".jpg"
#                         cv2.imwrite(path, face_unknown)
#                         c.execute("insert into people_inroom values ('%s', 1, 12)" % name)
#                         people.commit()
#             print(len(face_encodings_unknown_in_room))
            # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
        print("total in people in room ", total_in_room)
        print("total in people known name in room ", total_known_in_room)
    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
people.close()

Dam Van Tai
total in people in room  1
total in people known name in room  1
Dam Van Tai
total in people in room  1
total in people known name in room  1
Dam Van Tai
total in people in room  1
total in people known name in room  1
Dam Van Tai
total in people in room  1
total in people known name in room  1
Dam Van Tai
total in people in room  1
total in people known name in room  1
Dam Van Tai
total in people in room  1
total in people known name in room  1
Dam Van Tai
total in people in room  1
total in people known name in room  1
Dam Van Tai
total in people in room  1
total in people known name in room  1
Dam Van Tai
total in people in room  1
total in people known name in room  1
Dam Van Tai
total in people in room  1
total in people known name in room  1
Dam Van Tai
total in people in room  1
total in people known name in room  1
Dam Van Tai
total in people in room  1
total in people known name in room  1
Dam Van Tai
total in people in room  1
total in people known name in room  1

ha thi quynh huong
total in people in room  4
total in people known name in room  2
Dam Van Tai
total in people in room  4
total in people known name in room  2
Dam Van Tai
total in people in room  4
total in people known name in room  2
Dam Van Tai
total in people in room  4
total in people known name in room  2
Dam Van Tai
total in people in room  4
total in people known name in room  2
Dam Van Tai
total in people in room  4
total in people known name in room  2
Dam Van Tai
total in people in room  4
total in people known name in room  2
Dam Van Tai
total in people in room  4
total in people known name in room  2
Dam Van Tai
total in people in room  4
total in people known name in room  2
total in people in room  4
total in people known name in room  2
gia tri index 2
unknown_2
total in people in room  5
total in people known name in room  2
Dam Van Tai
total in people in room  5
total in people known name in room  2
Dam Van Tai
total in people in room  5
total in people known name i

KeyboardInterrupt: 